In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Dropout, Dense
from tensorflow.keras.models import Model
import numpy as np 
import pandas as pd
import os
import re

import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from keras.preprocessing.text import Tokenizer

from keras.layers import Input, Dense, Dropout, Embedding, LSTM, Conv1D, MaxPooling1D, Bidirectional, Concatenate, Reshape
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences



from sklearn.model_selection import train_test_split



In [6]:
# Load the dataset
data = pd.read_csv('/kaggle/input/answerscript2/p1.csv')

# Keep only the essay and domain1_score columns
data = data[['essay', 'domain1_score']]

# Remove rows with missing values
data = data.dropna()

# Define the maximum length of the essay (in words)
MAX_ESSAY_LENGTH = 500

# Define the maximum number of sentences per essay
MAX_SENTENCES = 50

# Define the maximum length of each sentence (in words)
MAX_SENTENCE_LENGTH = 20

# Preprocess the essays
def preprocess_essay(essay):
    # Convert to lowercase
    essay = essay.lower()
    # Remove non-alphabetic characters
    essay = re.sub("[^a-z\s+]","",essay)
    # Split the essay into sentences
    sentences = essay.split('.')
    # Remove leading/trailing white space from each sentence
    sentences = [s.strip() for s in sentences]
    # Remove empty sentences
    sentences = [s for s in sentences if len(s) > 0]
    # Truncate the essay to the maximum number of sentences
    sentences = sentences[:MAX_SENTENCES]
    # Tokenize each sentence
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sentences)
    sequences = tokenizer.texts_to_sequences(sentences)
    # Pad the sequences to the maximum sentence length
    sequences = pad_sequences(sequences, maxlen=MAX_SENTENCE_LENGTH)
    # Pad the sentences to the maximum number of sentences
    num_pad_sentences = MAX_SENTENCES - len(sentences)
    padded_sequences = pad_sequences([sequences], maxlen=MAX_SENTENCES, padding='post', truncating='post')
    if num_pad_sentences > 0:
        padded_sequences = np.append(padded_sequences, np.zeros((num_pad_sentences, MAX_SENTENCE_LENGTH)), axis=0)
    return padded_sequences[0]

def preprocess_text(x, remove_stopwords=False):
    x = x.lower()
    x = re.sub("[^a-z\s+]","",x)
    if remove_stopwords:
        x = " ".join([word for word in x.split() if word not in stopwords.words('english')])
    return x

# Apply the preprocess_essay function to each essay in the dataset
data['essay'] = data['essay'].apply(preprocess_text, remove_stopwords=True)

# Split the data into training, validation, and test sets
X = np.array(data['essay'].tolist())
y = np.array(data['domain1_score'].tolist())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [2]:
df.head()

NameError: name 'df' is not defined

In [7]:
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Dense, TimeDistributed, Dropout

# Define the input shape for each sentence
sentence_input_shape = (MAX_SENTENCE_LENGTH,)

# Define the input shape for each essay
essay_input_shape = (MAX_SENTENCES, MAX_SENTENCE_LENGTH)

# Define the number of LSTM units
NUM_LSTM_UNITS = 256

# Define the dropout rate
DROPOUT_RATE = 0.5
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=20)
tokenizer.fit_on_texts(data['essay'])

# Define the vocabulary size
VOCAB_SIZE = len(tokenizer.word_index) + 1

# Define the embedding dimension
EMBEDDING_DIM = 100



In [8]:
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Dropout, Lambda, TimeDistributed


In [9]:
df = pd.read_csv('/kaggle/input/answerscript2/p1.csv')

In [10]:
df_ind = df.loc[df['essay_set']==1]

df_ind = df_ind[['essay','domain1_score']]

df_ind = df_ind.sort_values(by= ["domain1_score"], ascending=False)
answer_sheet = df_ind.iloc[0]['essay']
df_ind = df_ind.drop(0)
students_answers = list(df['essay'].values)
marks_org = list(df['domain1_score'].values)

In [13]:
# Define the number of words to consider in the vocabulary
NUM_WORDS = 20000

# Define the maximum sequence length
MAX_SEQUENCE_LENGTH = 500

# Define the sentence input shape
sentence_input_shape = (MAX_SEQUENCE_LENGTH,)

texts = [essay for essay in data['essay']]
# Create the tokenizer
tokenizer = Tokenizer(num_words=NUM_WORDS)
tokenizer.fit_on_texts(texts)

# Define the vocabulary size
VOCAB_SIZE = len(tokenizer.word_index) + 1

# Define the embedding dimension
EMBEDDING_DIM = 100


In [14]:
data = data[~data['domain1_score'].isna()]

data['essay'] = data['essay'].apply(preprocess_text, remove_stopwords=True)
data['expected']=answer_sheet
train_df, test_df = train_test_split(data, test_size=0.1, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)




X_expected_train = pad_sequences(tokenizer.texts_to_sequences(train_df['expected']), maxlen=MAX_SEQUENCE_LENGTH)
X_actual_train = pad_sequences(tokenizer.texts_to_sequences(train_df['essay']), maxlen=MAX_SEQUENCE_LENGTH)
y_train = train_df['domain1_score'].values / 12

X_expected_val = pad_sequences(tokenizer.texts_to_sequences(val_df['expected']), maxlen=MAX_SEQUENCE_LENGTH)
X_actual_val = pad_sequences(tokenizer.texts_to_sequences(val_df['essay']), maxlen=MAX_SEQUENCE_LENGTH)
y_val = val_df['domain1_score'].values / 12

X_expected_test = pad_sequences(tokenizer.texts_to_sequences(test_df['expected']), maxlen=MAX_SEQUENCE_LENGTH)
X_actual_test = pad_sequences(tokenizer.texts_to_sequences(test_df['essay']), maxlen=MAX_SEQUENCE_LENGTH)
y_test = test_df['domain1_score'].values / 12

In [21]:
# Import the required libraries
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dropout, Dense, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Dropout, Lambda, TimeDistributed

# Define the list of texts
texts = [essay for essay in data['essay']]

# Define the number of words to consider in the vocabulary
NUM_WORDS = 20000

# Define the maximum sequence length
MAX_SEQUENCE_LENGTH = 500

# Define the sentence input shape
sentence_input_shape = (MAX_SEQUENCE_LENGTH,)

# Create the tokenizer
tokenizer = Tokenizer(num_words=NUM_WORDS)
tokenizer.fit_on_texts(texts)

# Define the vocabulary size
VOCAB_SIZE = len(tokenizer.word_index) + 1

# Define the embedding dimension
EMBEDDING_DIM = 100

# Define the sentence-level model
sentence_input = Input(shape=sentence_input_shape, dtype='int32', name='sentence_input')
sentence_embedding = Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH, name='sentence_embedding')(sentence_input)
lstm_layer = LSTM(NUM_LSTM_UNITS, return_sequences=True)
lstm_output = lstm_layer(sentence_embedding)
dropout_layer = Dropout(DROPOUT_RATE)
dropout_output = dropout_layer(lstm_output)
sentence_prediction = TimeDistributed(Dense(1, activation='sigmoid'))(dropout_output)

# Define the document-level model
document_input = Input(shape=(None, 500))
#document_input = Input(shape=(None, MAX_SEQUENCE_LENGTH), dtype='int32', name='document_input')
#doc_embedding = TimeDistributed(Lambda(lambda x: sentence_prediction(x)))(document_input)
#doc_embedding = TimeDistributed(Lambda(lambda x: sentence_prediction(tf.keras.backend.squeeze(x, axis=2))))(document_input)
doc_embedding = TimeDistributed(Lambda(lambda x: sentence_prediction(tf.squeeze(x, axis=1))))(document_input)

#doc_embedding = TimeDistributed(sentence_prediction)(document_input)
doc_lstm_layer = LSTM(NUM_LSTM_UNITS)
doc_lstm_output = doc_lstm_layer(doc_embedding)
doc_dropout_layer = Dropout(DROPOUT_RATE)
doc_dropout_output = doc_dropout_layer(doc_lstm_output)
document_prediction = Dense(1, activation='sigmoid')(doc_dropout_output)

# Define the model inputs and outputs
model_inputs = [document_input]
model_outputs = [document_prediction]

# Create the model
model = Model(inputs=model_inputs, outputs=model_outputs)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


# Train the model
model.fit(x=X_train, y=y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(X_val, y_val))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)


ValueError: Exception encountered when calling layer 'lambda_2' (type Lambda).

Can not squeeze dim[1], expected a dimension of 1, got 500 for '{{node time_distributed_8/lambda_2/Squeeze}} = Squeeze[T=DT_FLOAT, squeeze_dims=[1]](time_distributed_8/Reshape)' with input shapes: [?,500].

Call arguments received by layer 'lambda_2' (type Lambda):
  • inputs=tf.Tensor(shape=(None, 500), dtype=float32)
  • mask=None
  • training=None

NameError: name 'model' is not defined

In [ ]:
# Train the model
model.fit(x=X_train, y=y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(X_val, y_val))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

In [15]:
# Define the embedding dimension
EMBEDDING_DIM = 100

# Define the sentence-level model

sentence_input = Input(shape=sentence_input_shape, dtype='int32', name='sentence_input')
sentence_embedding = Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM, input_length=MAX_SENTENCE_LENGTH)(sentence_input)
sentence_lstm = LSTM(NUM_LSTM_UNITS)(sentence_embedding)
sentence_dropout = Dropout(DROPOUT_RATE)(sentence_lstm)

exp_sentence_input = Input(shape=sentence_input_shape, dtype='int32', name='exp_sentence_input')
exp_sentence_embedding = Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM, input_length=MAX_SENTENCE_LENGTH)(exp_sentence_input)
exp_sentence_lstm = LSTM(NUM_LSTM_UNITS)(exp_sentence_embedding)
exp_sentence_dropout = Dropout(DROPOUT_RATE)(exp_sentence_lstm)
model_inputs = [exp_sentence_input, sentence_input]
sentence_model = Model(inputs=model_inputs, outputs=sentence_dropout)

# Define the document-level model
#document_input = Input(shape=(None, 500))
document_input = Input(shape=(None, MAX_SEQUENCE_LENGTH), dtype='int32', name='document_input')
#document_input = Input(shape=document_input_shape, dtype='int32', name='document_input')
document_embedding = TimeDistributed(sentence_model)(document_input)
document_lstm = LSTM(NUM_LSTM_UNITS)(document_embedding)
document_dropout = Dropout(DROPOUT_RATE)(document_lstm)
document_output = Dense(1, activation='sigmoid')(document_dropout)

exp_document_input = Input(shape=(None, MAX_SEQUENCE_LENGTH), dtype='int32', name='exp_document_input')
#document_input = Input(shape=document_input_shape, dtype='int32', name='document_input')
exp_document_embedding = TimeDistributed(sentence_model)(exp_document_input)
exp_document_lstm = LSTM(NUM_LSTM_UNITS)(exp_document_embedding)
exp_document_dropout = Dropout(DROPOUT_RATE)(exp_document_lstm)
exp_document_output = Dense(1, activation='sigmoid')(exp_document_dropout)
model_inputs2 = [exp_document_input, document_input]
document_model = Model(inputs=model_inputs2, outputs=document_output)

# Compile the model
document_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse', 'mae'])

BATCH_SIZE = 64
EPOCHS = 10
# Train the model
'''document_model.fit(x=X_train, y=y_train, validation_data=(X_val, y_val), batch_size=BATCH_SIZE, epochs=EPOCHS)

# Evaluate the model
loss, mse, mae = document_model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print('Test Loss:', loss)
print('Test MSE:', mse)
print('Test MAE:', mae)'''


# Compile the model with MAE as the loss function

#model = Model(inputs=model_inputs, outputs=model_outputs)

# Compile the model with MAE as the loss function
#model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mae'])
document_model.fit(x=[X_expected_train, X_actual_train], y=y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=([X_expected_val, X_actual_val], y_val))

# Evaluate the model
loss, mae = document_model.evaluate([X_expected_test, X_actual_test], y_test, batch_size=BATCH_SIZE)
print('Test Loss:', loss)
print('Test MAE:', mae)

ValueError: Exception encountered when calling layer "time_distributed" (type TimeDistributed).

Layer "model" expects 2 input(s), but it received 1 input tensors. Inputs received: [<tf.Tensor 'time_distributed/Reshape:0' shape=(None, 500) dtype=int32>]

Call arguments received by layer "time_distributed" (type TimeDistributed):
  • inputs=tf.Tensor(shape=(None, None, 500), dtype=int32)
  • training=None
  • mask=None

In [4]:
!pip install Keras-Preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.9 MB/s eta 0:00:00
